In [1]:
import pandas as pd
d = {"headers":{"Student":["name","continent"]},"rows":{"Student":[["Jane","America"],["Pascal","Europe"],["Xi","Asia"],["Jack","America"]]}}
pd.DataFrame(d['rows']['Student'], columns=d['headers']['Student']).to_csv("./Student.txt", index=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("student report by geography").config("pyspark.sql.shuffle.partition","4").getOrCreate()

In [3]:
s = spark.read.option("header", True).csv("./Student.txt")
s.show()

+------+---------+
|  name|continent|
+------+---------+
|  Jane|  America|
|Pascal|   Europe|
|    Xi|     Asia|
|  Jack|  America|
+------+---------+



A school has students from Asia, Europe, and America.

Write an SQL query to pivot the continent column in the Student table so that each name is sorted alphabetically and displayed underneath its corresponding continent. The output headers should be America, Asia, and Europe, respectively.

The test cases are generated so that the student number from America is not less than either Asia or Europe.

The query result format is in the following example.

In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

Window_spec = Window.orderBy("name")
a = s.where("continent = 'America'").select(s.name.alias("America"), row_number().over(Window_spec).alias("rn"))
b = s.where("continent = 'Asia'").select(s.name.alias("Asia"), row_number().over(Window_spec).alias("rn"))
c = s.where("continent = 'Europe'").select(s.name.alias("Europe"), row_number().over(Window_spec).alias("rn"))

a.join(b, "rn", "leftouter").join(c, "rn", "leftouter").select("America", "Asia", 'Europe').show()

22/07/10 22:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/10 22:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/10 22:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----+------+
|America|Asia|Europe|
+-------+----+------+
|   Jack|  Xi|Pascal|
|   Jane|null|  null|
+-------+----+------+

